In [4]:
## first time series test
import sys
sys.path.append('../../')
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [5]:
from PIL import Image, ImageDraw, ImageFont
import os
# import cv2
import numpy as np

In [6]:
# load the data

WIDTH=18158
nlines = 7932

Image.MAX_IMAGE_PIXELS = WIDTH*nlines

CSK_RMLI_ras = ['/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200910.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200919.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200926.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200927.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201005.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201012.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201013.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201113.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201114.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201122.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201224.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201231.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20210101.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20210109.rmli.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20211101.rmli.ras']

TSX_RMLI_ras = ['/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201116.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201127.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201208.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201219.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210110.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210201.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210223.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210317.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210408.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210430.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210522.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210602.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20210613.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20211001.pwr.ras',
                '/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20211023.pwr.ras']



In [7]:
# load image form file
def load_imag(filename):
    with Image.open(filename) as image_pil:
        image = np.asarray(image_pil)
    return image

# normalise to 0 - 1
def normalise_arr(arr):
    """Normalises N-D numpy array to values between 0 and 1 where the minimum value 
       in the original array becomes 0 and the maximum value becomes 1.

    Args:
        arr (Numpy array): ND numeric array
    """
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))

# compare images
def compare_RMLI(file1, file2):
    image1 = load_imag(file1)
    image2 = load_imag(file2)
    # caluclate difference
    image_diff = image2 - image1

    # normalise
    RMLI1_arr_norm = normalise_arr(image1)
    RMLI2_arr_norm = normalise_arr(image2)
    RMLI_diff_arr_norm = normalise_arr(image_diff)

    # create rgb composite
    rgb_image = np.dstack([RMLI1_arr_norm,
                       RMLI2_arr_norm,
                       RMLI_diff_arr_norm])
    
    return image1, image2, rgb_image

In [13]:
# load images and crop

CSK_rmli1, CSK_rmli2, CSK_rgb = compare_RMLI(CSK_RMLI_ras[0],CSK_RMLI_ras[13])
TSX_rmli1, TSX_rmli2, TSX_rgb = compare_RMLI(TSX_RMLI_ras[0],TSX_RMLI_ras[3])

# crop
CSK_rmli1_crop = CSK_rmli1[4000:6000,1500:3000]
CSK_rmli2_crop = CSK_rmli2[4000:6000,1500:3000]
CSK_rgb_crop   = CSK_rgb  [4000:6000,1500:3000]

TSX_rmli1_crop = TSX_rmli1[4000:8000,1000:3000]
TSX_rmli2_crop = TSX_rmli2[4000:8000,1000:3000]
TSX_rgb_crop   = TSX_rgb  [4000:8000,1000:3000]


In [14]:
%matplotlib osx
fig = plt.figure(figsize=(8,8))

gs = GridSpec(2, 3, figure=fig,height_ratios=[10,20])

ax00 = fig.add_subplot(gs[0, 0]) # image1
ax01 = fig.add_subplot(gs[0, 1]) # image2
ax02 = fig.add_subplot(gs[0, 2]) # composite

ax10 = fig.add_subplot(gs[1, 0]) # image1
ax11 = fig.add_subplot(gs[1, 1]) # image2
ax12 = fig.add_subplot(gs[1, 2]) # composite

bbox = dict(fc="1.0",alpha=0.5)
cmaps = ['Greys_r', 'Greys_r','None','Greys_r', 'Greys_r','None']
labels = ['A','B','C','D','E','F']
label_x = [0.02,0.02,0.02,0.02,0.02,0.02]
label_y = [0.91,0.91,0.91,0.955,0.955,0.955]
for i, (a,image,cmap,label,lab_x,lab_y) in enumerate(zip([ax00,ax01,ax02,ax10,ax11,ax12],[CSK_rmli1_crop, CSK_rmli2_crop, CSK_rgb_crop,TSX_rmli1_crop, TSX_rmli2_crop, TSX_rgb_crop],cmaps,labels,label_x,label_y)):
    print(i)
    if cmap == 'None':
        a.imshow(image)
    else:
        a.imshow(image,cmap=cmap)
    
    a.annotate(label,xy=[lab_x,lab_y],xycoords='axes fraction',fontsize=16, bbox=bbox)
    a.annotate
    a.set_axis_off()

fig.tight_layout()

0
1
2
3
4
5
